# PSI运行示例

以下示例仅用于说明PSI的工作流程，目前仅支持单机测试。

加载数据集

In [1]:
import pandas as pd
import os
project_dir = os.path.dirname(os.path.abspath(''))
data_dir = os.path.join(project_dir, 'Datasets', 'data', 'data')
host_data = pd.read_csv(os.path.join(data_dir, 'breast_hetero_host.csv'))
guest_data = pd.read_csv(os.path.join(data_dir, 'breast_hetero_guest.csv'))
print("Host Data:")
print(host_data.head())

Host Data:
   id         x0        x1        x2        x3        x4        x5         x6  \
0   0  13.406929  1.396708  8.951264 -6.391955 -0.907567  6.952439  15.412624   
1   1 -15.119118 -6.659451 -3.972570 -6.783692  2.288610 -8.214613  -1.170056   
2   2  -2.344736 -9.145379  6.902479  8.605027  7.654483 -5.199578  10.779235   
3   3   7.357845  7.224143  7.794408 -5.304755  2.077003  4.312937   4.674460   
4   4   6.915768  3.606743 -2.804147  1.889518 -3.185365 -1.995137   9.173883   

          x7         x8  ...       x20       x21        x22       x23  \
0  11.820768   7.576128  ... -3.284186 -5.337557   1.408300  8.788250   
1  -4.004517  -6.820910  ... -9.234468 -0.150261  -5.890043 -8.587167   
2  -1.351300 -11.478583  ...  1.245020  5.234590   4.306623 -6.161988   
3   5.258720   7.626586  ... -1.544057 -4.316854  -3.344796  0.530144   
4   3.600453   6.651459  ... -3.524715  1.349833  12.310861 -4.455094   

         x24        x25        x26       x27        x28       x

服务端、客户端加载数据

In [2]:
from PSI import PSICompany, PSIPartner

company_key, company_features = host_data['id'], host_data.drop(columns=['id'])
partner_key, partner_features = guest_data['id'], guest_data.drop(columns=['id'])
company_key = company_key.astype(str)
partner_key = partner_key.astype(str)
company = PSICompany(company_key, company_features)
partner = PSIPartner(partner_key, partner_features)


[2025-08-12 21:41:46.397] [info] [thread_pool.cc:30] Create a fixed thread pool with size 31


服务端加密并交换公钥

In [3]:
U_c, company_pk = company.exchange()
# 将服务端公钥发送给客户端（TODO）

客户端加密、交换公钥、重排服务端数据

In [4]:
E_c, U_p, partner_pk = partner.exchange(U_c, company_pk)
# 将客户端公钥发送给服务端（TODO）

Computing masked company cipher


服务端计算交集，输出交集碎片

In [5]:
L, R_cI = company.compute_intersection(E_c, U_p, partner_pk)
# 将L和R_cI发送给客户端（TODO）

Computing masked partner cipher
Computing company shares


客户端输出交集碎片

In [6]:
R_pI = partner.output_shares(L)

Computing partner shares


获取交集碎片

注意：Company所持有属性在share矩阵左边，Partner所持有属性在右边。

In [7]:
company_share = R_cI[0]
partner_share = R_pI[0]


恢复并打印交集

In [8]:
R_I = company_share + partner_share
print(R_I)

[[  4.482732   7.606706   8.232204 ...  -0.068126  -9.833931   0.      ]
 [ 10.212381  -0.151352  12.701536 ...  -7.421358  -5.325319   1.      ]
 [  3.180015   6.39739    7.509831 ...  -3.4305    -0.523406   0.      ]
 ...
 [  5.743327   0.06047    2.378849 ...   9.842905 -13.84182    0.      ]
 [ 13.039767  -5.216417   3.895059 ...   1.43436   -3.497174   1.      ]
 [ -8.576518  -4.080317   0.216021 ...   1.195891  -3.983099   0.      ]]
